In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
import pandas as pd
import numpy as np

In [3]:
# https://www.kaggle.com/datasets/namanj27/ner-dataset
df = pd.read_csv("/content/drive/Shareddrives/CapBatu/ML Divsion/Named Entity Recognition Location/data/raw/ner_datasetreference converted.csv", engine='python')
df = df.fillna(method="ffill")

df

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,Sentence: 1,of,IN,O
2,Sentence: 1,demonstrators,NNS,O
3,Sentence: 1,have,VBP,O
4,Sentence: 1,marched,VBN,O
...,...,...,...,...
1048570,Sentence: 47959,they,PRP,O
1048571,Sentence: 47959,responded,VBD,O
1048572,Sentence: 47959,to,TO,O
1048573,Sentence: 47959,the,DT,O


In [11]:
np.sort(df["Tag"].unique())
# See

array(['B-art', 'B-eve', 'B-geo', 'B-gpe', 'B-nat', 'B-org', 'B-per',
       'B-tim', 'I-art', 'I-eve', 'I-geo', 'I-gpe', 'I-nat', 'I-org',
       'I-per', 'I-tim', 'O'], dtype=object)

In [12]:
df[df["Tag"] == "B-geo"]

,Sentence #,Word,POS,Tag
6,Sentence: 1,London,NNP,B-geo
12,Sentence: 1,Iraq,NNP,B-geo
65,Sentence: 3,Hyde,NNP,B-geo
94,Sentence: 5,Britain,NNP,B-geo
106,Sentence: 5,Brighton,NNP,B-geo
...,...,...,...,...
1048181,Sentence: 47941,southern,JJ,B-geo
1048221,Sentence: 47943,Burma,NNP,B-geo
1048456,Sentence: 47953,Bermel,NNP,B-geo
1048459,Sentence: 47953,Paktika,NNP,B-geo


In [13]:
df[df["Tag"] == "I-geo"]
# We'are going to create a detection on Tag "B-geo" and "I-geo" only. Because we want a location
# Because what we're seen that "Hyde Park" the "Hyde" is B-geo and "Park" is I-geo
# We also want to tokenize each sentences
# We're going to create uncased BiLSTM Model

# So the model is sequence to sequence model.
# With input a word, and ouput a 0 or 1 representing tag of B-geo
# Maybe train with sliding window dataset?

,Sentence #,Word,POS,Tag
66,Sentence: 3,Park,NNP,I-geo
347,Sentence: 15,State,NNP,I-geo
350,Sentence: 15,State,NNP,I-geo
381,Sentence: 17,Delta,NNP,I-geo
561,Sentence: 24,Arab,NNP,I-geo
...,...,...,...,...
1047365,Sentence: 47898,India,NNP,I-geo
1047948,Sentence: 47926,Afghanistan,NNP,I-geo
1048026,Sentence: 47930,City,NNP,I-geo
1048073,Sentence: 47934,Afghanistan,NNP,I-geo


Preprocessing Text and Domain Interest

In [14]:
import re
import numpy as np
from typing import List

def remove_space_start_end(text:str):
  return text.strip()

def lowercase(text:str):
  return text.lower()

def only_alphabets(text:str):
  text = re.sub("[\.]", "", text)
  text = re.sub('[^a-zA-Z]',' ',text) 
  text = re.sub(' (rd|th|st) '," ", text)
  return text

def remove_multiple_space(text:str):
  return " ".join(text.split())

def preprocessText(text:str):
  text = str(text)
  text = lowercase(text)
  text = only_alphabets(text)
  text = remove_multiple_space(text)
  if len(text.split()) == 0:
    return np.nan
  return text

def preprocessTag(text:str):
  # 'B-eve', 'B-gpe', 'B-nat'
  text = str(text)
  if text == "B-geo" or text == "I-geo":
    return 1
  return 0

In [15]:
dummy_df = df.copy()
dummy_df["Word"] = dummy_df["Word"].apply(preprocessText)
dummy_df["Tag"] = dummy_df["Tag"].apply(preprocessTag)
dummy_df

,Sentence #,Word,POS,Tag
0,Sentence: 1,thousands,NNS,0
1,Sentence: 1,of,IN,0
2,Sentence: 1,demonstrators,NNS,0
3,Sentence: 1,have,VBP,0
4,Sentence: 1,marched,VBN,0
...,...,...,...,...
1048570,Sentence: 47959,they,PRP,0
1048571,Sentence: 47959,responded,VBD,0
1048572,Sentence: 47959,to,TO,0
1048573,Sentence: 47959,the,DT,0


In [16]:
dummy_df.isna().sum()

Sentence #         0
Word          103476
POS                0
Tag                0
dtype: int64

In [17]:
dummy_df = dummy_df.dropna(subset=["Word"])
dummy_df = dummy_df.reset_index(drop=True)
dummy_df = dummy_df.drop("POS", axis=1)

In [18]:
dummy_df.isna().sum()

Sentence #    0
Word          0
Tag           0
dtype: int64

In [19]:
data = {
    "word": dummy_df.groupby("Sentence #")["Word"].apply(list).to_list(),
    "tag": dummy_df.groupby("Sentence #")["Tag"].apply(list).to_list()
}

final_df = pd.DataFrame(data)
final_df.astype('object')

,word,tag
0,"[thousands, of, demonstrators, have, marched, ...","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, ..."
1,"[iranian, officials, say, they, expect, to, ge...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"[helicopter, gunships, saturday, pounded, mili...","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
3,"[they, left, after, a, tense, hour long, stand...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
4,"[un, relief, coordinator, jan, egeland, said, ...","[1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...
47953,"[opposition, leader, mir, hossein, mousavi, ha...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
47954,"[on, thursday, iranian, state, media, publishe...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
47955,"[following, iran, s, disputed, june, elections...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
47956,"[since, then, authorities, have, held, public,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [23]:
# drop a sentence that doesnt have 1 in tag column
row_idx = []
for i in range(len(final_df)):
  tag = final_df["tag"].iloc[i]
  if 1 not in tag:
    row_idx.append(i)

print(row_idx)
final_df = final_df.drop(row_idx)

[1, 3, 7, 11, 14, 15, 16, 17, 18, 19, 20, 21, 25, 26, 27, 29, 30, 31, 34, 36, 37, 38, 39, 41, 43, 45, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 61, 62, 63, 65, 66, 68, 69, 72, 75, 76, 77, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 92, 93, 98, 99, 102, 103, 105, 107, 110, 111, 112, 114, 115, 116, 117, 120, 121, 122, 125, 126, 127, 128, 129, 130, 131, 134, 136, 139, 140, 141, 143, 145, 146, 147, 149, 150, 152, 155, 158, 160, 161, 162, 163, 164, 165, 166, 170, 172, 173, 174, 176, 177, 178, 179, 180, 181, 183, 184, 185, 186, 187, 188, 189, 190, 192, 193, 194, 195, 196, 197, 198, 199, 203, 207, 208, 210, 211, 212, 214, 215, 216, 218, 220, 222, 223, 224, 225, 226, 229, 230, 231, 232, 233, 234, 238, 241, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 255, 256, 260, 261, 263, 266, 267, 268, 269, 270, 272, 274, 276, 279, 280, 282, 283, 285, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 303, 305, 307, 308, 309, 311, 319, 320, 323, 324, 325, 328, 329, 330

In [24]:
for i in range(len(final_df)):
  words = final_df["word"].iloc[i]
  tag = final_df["tag"].iloc[i]
  if len(words) != len(tag):
    print(f"Found on {i}")

# No printed logs, thus
# All words have each tag

In [ ]:
final_df.to_csv("/content/drive/Shareddrives/CapBatu/ML Divsion/Named Entity Recognition Location/data/processed/ner_location.csv", index=False)